# Summary 
Demonstrates calculation of 50/10 access levels from the NBD map data against arbitrary 
geometry other than the precomputed hexagons.

Calculating this requires joining the geometry from the Canada wide hexagons geometry 
with the tabular `PHH_Speeds_Current-PHH_Vitesses_Actuelles.csv` dataset. 

Loading the data and executing the spatial joins takes a total of 7 minutes.

In [1]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 1
%aimport src.datasets.joins
%aimport src.datasets.loading.statcan

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from src.datasets.loading import statcan
from src.datasets.loading import ookla
from src.datasets import overlays

from src.datasets import joins
import src

In [3]:
custom_areas = joins.hexagons_popctrs_removed()

/Users/zacharyshand/ookla-statcan-analysis/notebooks/../src/datasets/overlays.py:25: UserWarning: `keep_geom_type=True` in overlay resulted in 418 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  ol = gp.overlay(left, right, how="union")
/Users/zacharyshand/opt/anaconda3/envs/datascience/lib/python3.10/site-packages/geopandas/geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [4]:
phh_speed_csv = src.config.DATA_DIRECTORY / 'PHH' / 'CRTC_Speed_Data' / 'PHH_Speeds_Current-PHH_Vitesses_Actuelles.csv'
phh_speeds = pd.read_csv(phh_speed_csv)

phh_geo = statcan.phh_geometry()

phh = phh_geo.merge(phh_speeds, on='PHH_ID')
phh = phh.merge(statcan.phh_data(), on='PHH_ID')
phh = phh.loc[lambda s:s.Type !=8] # drop PHH nulls

In [6]:
o = joins.add_phh_pop(custom_areas, phh, 'HEXUID_PCPUID')

In [9]:
del o['HEXuid_HEXidu'] # we don't want these, they're quite long in large cities

In [11]:
popctrs = statcan.boundary('population_centres') # use this to generate a filtering region around cities

In [12]:
#Can't plot the whole country, too many hexagons.
xmin, ymin, xmax, ymax = popctrs.loc[lambda s:s.PCNAME=="Kelowna"].buffer(100000).total_bounds
o.cx[xmin:xmax,ymin:ymax].loc[lambda s:(s['Pop2016'] > 0) | (s['TDwell2016_TLog2016'] > 0) | (s['URDwell2016_RH2016']>0)].explore('Pop_Avail_50_10')